In [8]:
# predict: source: https://surprise.readthedocs.io/en/stable/FAQ.html
from collections import defaultdict
from tkinter import *
import json
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from tkinter import *

def read_data():
    rent_list = list()
    category_set=set()
    with open('renttherunway_final_data.json') as json_data:
        # read all data in
        for line in json_data:
            rent_entry = json.loads(line.strip())
            # uid, iid, rating: we only keep those lines that have ratings.
            # This is called sparse-matrix form
            rent_extracted_entry = dict()
            if rent_entry['rating']:
                rent_extracted_entry['user_id'] = rent_entry['user_id']
                rent_extracted_entry['item_id'] = rent_entry['item_id']
                rent_extracted_entry['rating'] = float(rent_entry['rating'])
                # first filter data according to users' choice: category
                rent_extracted_entry['category'] = rent_entry['category']
                category_set.add(rent_entry['category'])
                rent_list.append(rent_extracted_entry)
    """
    print("Category:")
    print(category_set)
    print(len(rent_list))
    """
    return rent_list

def filter_needs(rent_list, category):
    # return a dataframe, each line <uid, iid, rating> (raw values: strings)
    rent_df = pd.DataFrame()
    for entry in rent_list:
        if entry['category'] == category:
            rent_df = rent_df.append(entry, ignore_index=True)
    return rent_df[['user_id', 'item_id', 'rating']]

def grid_search(my_df):
    reader = Reader(rating_scale=(1, 10))
    data = Dataset.load_from_df(my_df, reader)

    param_grid = {'k': [1, 3, 5, 10, 15, 20],
                  'min_k': [1, 3, 5, 10],
                  'sim_options': {'name': ['pearson'],
                                  'min_support': [1, 3, 5, 10],
                                  'user_based': [True]}}
    # knnwithmeans decide the formula for prediction
    # sim_options decide the formula for calculating similarities
    # min_k: if there are not enough neighbors, prediction is the mean of all R_ui.
    # sim_options: min_support: min number of common items: if common items < min_support, sim(u, v)=0
    knn_grid_search = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=5)
    # data is training + validation, GridSearchCV separates training and validation for me
    # when fit, fit(trainval)
    # get test data through build_anti_testset: users and items are known, but ratings are unknown.
    knn_grid_search.fit(data)
    print(knn_grid_search.best_score['rmse'])
    print(knn_grid_search.best_params['rmse'])
    algo = knn_grid_search.best_estimator['rmse']
    print(algo)
    trainvalset = data.build_full_trainset()
    algo.fit(trainvalset)
    # Then predict ratings for all pairs (u, i) that are NOT in the training set.
    testset = trainvalset.build_anti_testset()
    predictions = algo.test(testset)
    return algo, trainvalset, predictions


def get_top_n(my_df, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''
    # predictions(list of Prediction objects): The list of predictions, as
    # returned by the test method of an algorithm.
    algo, trainvalset, predictions = grid_search(my_df)
    # First map the predictions to each user. A dictionary of lists.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n



In [11]:
#GUI for the Recommended result
import tkinter as tk
from tkinter import ttk

def show():

    #tempList = [['Jim', '0.33'], ['Dave', '0.67'], ['James', '0.67'], ['Eden', '0.5']]
    #tempList.sort(key=lambda e: e[1], reverse=True)

    #print(top_n['139671'])
    tempList = top_n['4040']
    print("This is show()")
    for i, (item, rating) in enumerate(tempList, start=1):
        listBox.insert("", "end", values=(i, item, rating))

def display():
    
    rating = tk.Tk() 
    label = tk.Label(rating, text="Best Clothing for You", font=("Arial",30)).grid(row=0, columnspan=3)
    # create Treeview with 3 columns
    cols = ('Ranking', 'Item ID', 'Rating')

    listBox = ttk.Treeview(rating, columns=cols, show='headings')
    # set column headings
    for col in cols:
        listBox.heading(col, text=col)    
    listBox.grid(row=1, column=0, columnspan=2)

    showScores = tk.Button(rating, text="Show scores", width=15, command=show).grid(row=4, column=0)
    closeButton = tk.Button(rating, text="Close", width=15, command=exit).grid(row=4, column=1)

    rating.mainloop()

In [12]:
def main():
    rent_list = read_data()

    # get category from user
    category = 'legging'
    #category = sel()
    my_df = filter_needs(rent_list, category)
    print(my_df)
    top_n = get_top_n(my_df, n=10)
    #print(top_n['139671'])
    display()


main()

   user_id  item_id  rating
0   139671  2064568    10.0
1   486996  2064568    10.0
2    81759  2064568    10.0
3   517457  2064568     6.0
4   881924  2064568    10.0
5   240639  2064568     8.0
6   836090  2064568    10.0
7   593308  2064568    10.0
8   525214  2397911    10.0
9   745064  2064568    10.0
10  649398  2064568    10.0
11  373867  2064568    10.0
12  171791  2064568    10.0
13  792209  2064568    10.0
14   58230  2064568    10.0
15  315261  2064568    10.0
16  182652  2064568    10.0
17  331029  2064568     8.0
18  993059  2656632    10.0
19  788096  2656632    10.0
20  649790  2064568     8.0
21  112076  2064568    10.0
22  219063  2064568    10.0
23  319583  2064568    10.0
24  742505  2064568     8.0
25  122798  2064568    10.0
26  679535  2656632    10.0
27  645325  2656632    10.0
28  504121  2397911     8.0
29   46566  2397911    10.0
..     ...      ...     ...
57  207896  2656632     8.0
58  391584  2064568    10.0
59  859574  2064568    10.0
60  569874  2397911 

Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda\lib\lib-tk\Tkinter.py", line 1547, in __call__
    return self.func(*args)
  File "<ipython-input-11-abef56a63fda>", line 11, in show
    tempList = top_n['4040']
NameError: global name 'top_n' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda\lib\lib-tk\Tkinter.py", line 1547, in __call__
    return self.func(*args)
  File "<ipython-input-11-abef56a63fda>", line 11, in show
    tempList = top_n['4040']
NameError: global name 'top_n' is not defined


In [7]:
import tkinter as tk


LARGE_FONT= ("Verdana", 12)
    

class SeaofBTCapp(tk.Tk):

    def __init__(self, *args, **kwargs):
        
        tk.Tk.__init__(self, *args, **kwargs)
        container = tk.Frame(self)

        container.pack(side="top", fill="both", expand = True)

        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}

        for F in (ChooseCategory, GetUserID, GetRecommendation):

            frame = F(container, self)

            self.frames[F] = frame

            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame(ChooseCategory)

    def show_frame(self, cont):

        frame = self.frames[cont]
        frame.tkraise()


        
class ChooseCategory(tk.Frame):

    def sel():
        #selection = "You selected " + str(v.get())
        selection = v.get()
        if selection == 1:
            selection = 'legging'
        else:
            selection = "down"

        return selection
    
    
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text="Choose Category Page", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        # All Categories
        v = tk.IntVar() 
        tk.Radiobutton(self, text='Legging', variable=v, value=1, command=sel).pack()
        tk.Radiobutton(root, text='Down', variable=v, value=2, command=sel).pack() 
        
        
        button = tk.Button(self, text="Get User ID", command=lambda: controller.show_frame(GetUserID))
        button.pack()

        """ should be in the GetUserID
        button2 = tk.Button(self, text="Get Result", command=lambda: controller.show_frame(GetResult))
        button2.pack()
        """


class GetUserID(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Enter User ID!", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = tk.Button(self, text="Back to Home", command=lambda: controller.show_frame(ChooseCategory))
        button1.pack()

        button2 = tk.Button(self, text="Get Recommendation", command=lambda: controller.show_frame(GetRecommendation))
        button2.pack()


class GetRecommendation(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Get the Recommendation!", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = tk.Button(self, text="Back to Home",
                            command=lambda: controller.show_frame(ChooseCategory))
        button1.pack()

        """
        button2 = tk.Button(self, text="Get UserID",
                            command=lambda: controller.show_frame(GetUserID))
        button2.pack()
        """


        
def main():
    rent_list = read_data()
    
    app = SeaofBTCapp()
    app.mainloop()

    #test1 = SeaofBTCapp(tk)
    # get category from user

    category = 'legging'
    category = test1.sel()
    
    my_df = filter_needs(rent_list, category)
    print(my_df)
    
    top_n = get_top_n(my_df, n=10)
    print(top_n['139671'])

main()
    

In [ ]:
def main():
    rent_list = read_data()

    # get category from user
    category = 'legging'
    #category = sel()
    my_df = filter_needs(rent_list, category)
    print(my_df)
    top_n = get_top_n(my_df, n=10)
    print(top_n['139671'])


main()

In [ ]:
"""

import tkinter as tk

LARGE_FONT= ("Verdana", 12)


class SeaofBTCapp(tk.Tk):

    def __init__(self, *args, **kwargs):
        
        tk.Tk.__init__(self, *args, **kwargs)
        container = tk.Frame(self)

        container.pack(side="top", fill="both", expand = True)

        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}

        for F in (ChooseCategory, GetUserID, GetResult):

            frame = F(container, self)

            self.frames[F] = frame

            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame(ChooseCategory)

    def show_frame(self, cont):

        frame = self.frames[cont]
        frame.tkraise()
    



class ChooseCategory(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text="Start Page", font=LARGE_FONT)
        label.pack(pady=10,padx=10)
        
        v = tk.IntVar() 
        tk.Radiobutton(self, text='Legging', variable=v, value=1).pack()

        button = tk.Button(self, text="Get User ID", command=lambda: controller.show_frame(GetUserID))
        button.pack()


        button2 = tk.Button(self, text="Get Result", command=lambda: controller.show_frame(GetResult))
        button2.pack()


class GetUserID(tk.Frame):
    
    def get_userID(self):
        print(str(e1.get()))

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="GetUserID", font=LARGE_FONT)
        label.pack(pady=10,padx=10)
        
                                       
        #master = tk.Tk()
        tk.Label(self, text="User ID").grid(row=0)

        e1 = tk.Entry(self)
        e1.grid(row=0, column=1)
        tk.Button(self, text='Quit', command=self.quit).grid(row=3, column=0, sticky=tk.W, pady=4)
        tk.Button(self, text='Enter', command=self.get_userID).grid(row=3, column=1, sticky=tk.W, pady=4)

        #tk.mainloop()
        
        
        button1 = tk.Button(self, text="Back to Home", command=lambda: controller.show_frame(ChooseCategory))
        button1.pack()

        button2 = tk.Button(self, text="Get Result", command=lambda: controller.show_frame(GetResult))
        button2.pack()




class GetResult(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Page Two!!!", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = tk.Button(self, text="Back to Home",
                            command=lambda: controller.show_frame(ChooseCategory))
        button1.pack()

        button2 = tk.Button(self, text="Page One",
                            command=lambda: controller.show_frame(PageOne))
        button2.pack()
        


app = SeaofBTCapp()
app.mainloop()
"""